In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [161]:
df1 = pd.read_csv('C://Users//user//Desktop//major_project//keystroke-dynamics-datagen-master//Keystroke-Dynamics-Using-LSTMs-master//Keystroke-Dynamics-Using-LSTMs-master/shiva_appended.csv')
df2= pd.read_csv('C://Users//user//Desktop//major_project//keystroke-dynamics-datagen-master//Keystroke-Dynamics-Using-LSTMs-master//Keystroke-Dynamics-Using-LSTMs-master/shiva_fraud_appended.csv')

In [162]:
#genuine
df1.shape

(1000, 32)

In [163]:
#fraud
df2.shape

(400, 32)

In [164]:
#output for genuine data
df1['output']=1

In [194]:
#output for fraud
df2['output']=0
df_train=pd.concat([df1, df2], axis=0)
from sklearn.utils import shuffle
df = shuffle(df_train)
from sklearn.utils import shuffle
df = shuffle(df_train)
#dropping unwanted columns
df=df.drop(['Result','Place','Current_checkin_time','Last_Checkin_time'], axis=1)
df.columns.size

29

In [166]:
#Random positive horizontal stacking of Genuine with Genuine 
data = df.as_matrix()
data[data[:, 0] == "shiva"]
temp_d = data[data[:, 0] == "shiva"]
n_pos_per_subject=150
poss = []
first_half = temp_d[np.random.choice(400, n_pos_per_subject), 1:-1]
second_half = temp_d[np.random.choice(400, n_pos_per_subject), 1:-1]
poss.append(np.hstack([first_half, second_half]))
poss=np.vstack(poss)

In [167]:
poss.shape

(150, 54)

In [168]:
#Random negative stacking of Genuine with Fraud
data = df.as_matrix()
temp_d = data[data[:, 0] == "shiva"]
temp_notd = data[data[:, 0] == "fshiva"]
n_pos_per_subject=150
negs = []
first_half = temp_d[np.random.choice(400, n_pos_per_subject), 1:-1]
second_half = temp_notd[np.random.choice(400, n_pos_per_subject), 1:-1]
negs.append(np.hstack([first_half, second_half]))
negs=np.vstack(negs)

In [169]:
negs.shape

(150, 54)

In [170]:
#labelling for both postive and negative data
labels = np.zeros(poss.shape[0] + negs.shape[0])
labels[:poss.shape[0]] = 1
labels = np.expand_dims(labels, axis=1)
labels.shape

(300, 1)

In [171]:
#combining positive nd negative data
all_data = np.hstack([np.vstack([poss, negs]), labels])
all_data.shape

(300, 55)

In [172]:
np.random.shuffle(all_data)

In [173]:
all_data_t = np.zeros((all_data.shape[0], 9, 6))

In [196]:
#Splitting the parameter for LSTM input
ctr = 0
for i, j in zip(range(0, 27, 3), range(27, 54, 3)):
    all_data_t[:, ctr, :] = np.hstack([all_data[:, i:i+3], all_data[:, j:j+3]])
    ctr += 1

In [197]:
all_data_t.shape

(300, 9, 6)

In [198]:
X, y = all_data_t, all_data[:, -1]
print(X.shape)
print(y.shape)
print(y[10:20])

(300, 9, 6)
(300,)
[1.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0]


In [199]:
!pip install keras

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [200]:

import keras
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector, Concatenate
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.models import Model
from keras.optimizers import Adam

In [201]:

VALIDATION_SPLIT = 0.1

INPUT_SHAPE = [None, 2]

BATCH_SIZE = 32

EPOCHS = 20

In [202]:
#Spltting the train test data
def train_dev_split(x, y, val_split=0.1):
    m = x.shape[0]
    val_size = int(0.1 * m)
    return x[:-val_size], y[:, :-val_size, :], x[-val_size:], y[:, -val_size:, :]

In [203]:
#converting into catergorical 
y = to_categorical(y)
print(y.shape)
print(y[0])
y_ = np.zeros((9, y.shape[0], y.shape[1]))
for i in range(9):
    y_[i, :, :] = y
print(y_.shape)

(300, 2)
[1. 0.]
(9, 300, 2)


In [204]:
y_[0, 0, :], y[0]

(array([1., 0.]), array([1., 0.], dtype=float32))

In [205]:
#Splitting the data
x_train, y_train, x_test, y_test = train_dev_split(X, y_)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((270, 9, 6), (9, 270, 2), (30, 9, 6), (9, 30, 2))

In [206]:
n_a = 10
n_out = 2

In [207]:
#LSTM model
reshapor = Reshape((1, 6))
LSTM_cell = LSTM(n_a, return_state = True)
densor = Dense(n_out, activation='softmax')

In [208]:
#LSTM model
def keystroke_model(Tx, n_in, n_a, n_out):
    X = Input(shape=(Tx, n_in))
    
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    outputs = []

    for t in range(Tx):
         
        x = Lambda(lambda x: X[:, t, :])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        print(a.shape)
        out = densor(a)
        #print(out.shape)
        outputs.append(out)

    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    return model

In [209]:
model = keystroke_model(9, 6, n_a, n_out)

(?, 10)
(?, 10)
(?, 10)
(?, 10)
(?, 10)
(?, 10)
(?, 10)
(?, 10)
(?, 10)


In [188]:
#Compiling
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [189]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 9, 6)         0                                            
__________________________________________________________________________________________________
lambda_29 (Lambda)              (None, 6)            0           input_7[0][0]                    
__________________________________________________________________________________________________
reshape_6 (Reshape)             (None, 1, 6)         0           lambda_29[0][0]                  
                                                                 lambda_30[0][0]                  
                                                                 lambda_31[0][0]                  
                                                                 lambda_32[0][0]            

In [190]:
m = x_train.shape[0]
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [191]:
#Training
model.fit([x_train, a0, c0], list(y_train), epochs=10)

Epoch 1/10
270/270 [==============================] - 4s 14ms/step - loss: 6.2740 - dense_6_loss: 0.7395 - dense_6_accuracy: 0.5148 - dense_6_accuracy_1: 0.5222 - dense_6_accuracy_2: 0.5259 - dense_6_accuracy_3: 0.5259 - dense_6_accuracy_4: 0.5296 - dense_6_accuracy_5: 0.5630 - dense_6_accuracy_6: 0.5333 - dense_6_accuracy_7: 0.5667 - dense_6_accuracy_8: 0.5000
Epoch 2/10
270/270 [==============================] - 0s 262us/step - loss: 5.8298 - dense_6_loss: 0.6242 - dense_6_accuracy: 0.5037 - dense_6_accuracy_1: 0.5222 - dense_6_accuracy_2: 0.5519 - dense_6_accuracy_3: 0.5370 - dense_6_accuracy_4: 0.5481 - dense_6_accuracy_5: 0.7963 - dense_6_accuracy_6: 0.6519 - dense_6_accuracy_7: 0.6519 - dense_6_accuracy_8: 0.6481
Epoch 3/10
270/270 [==============================] - 0s 273us/step - loss: 5.4673 - dense_6_loss: 0.6152 - dense_6_accuracy: 0.7926 - dense_6_accuracy_1: 0.7963 - dense_6_accuracy_2: 0.8667 - dense_6_accuracy_3: 0.8630 - dense_6_accuracy_4: 0.9259 - dense_6_accuracy_5: 

In [192]:
m_t = x_test.shape[0]
a0_t = np.zeros((m_t, n_a))
c0_t = np.zeros((m_t, n_a))

In [193]:
#Testing
model.evaluate([x_test, a0_t, c0_t], list(y_test))

30/30 [==============================] - 1s 28ms/step


[4.574329376220703,
 0.5950793027877808,
 0.45621857047080994,
 0.3842138648033142,
 0.5046903491020203,
 0.704274594783783,
 0.3703799545764923,
 0.45164865255355835,
 0.5945038199424744,
 0.5133202075958252,
 0.8666666746139526,
 0.8666666746139526,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.8666666746139526,
 0.8333333134651184,
 0.800000011920929,
 0.800000011920929]